### Importando os dados

In [78]:
import pandas as pd

# 1. Carregar o dataset limpo
df = pd.read_csv('Most Streamed Spotify Songs 2024.csv', encoding='latin-1')

# 2. Conversões e limpezas adicionais
df['Release Date'] = pd.to_datetime(df['Release Date'], errors='coerce')
df = df.dropna(subset=['Track', 'Artist', 'ISRC'])
df.head()

,Track,Album Name,Artist,Release Date,ISRC,All Time Rank,Track Score,Spotify Streams,Spotify Playlist Count,Spotify Playlist Reach,...,SiriusXM Spins,Deezer Playlist Count,Deezer Playlist Reach,Amazon Playlist Count,Pandora Streams,Pandora Track Stations,Soundcloud Streams,Shazam Counts,TIDAL Popularity,Explicit Track
0,MILLION DOLLAR BABY,Million Dollar Baby - Single,Tommy Richman,2024-04-26,QM24S2402528,1,725.4,"390,470,936","30,716","196,631,588",...,684,62.0,"17,598,718",114.0,"18,004,655","22,931","4,818,457","2,669,262",NaN,0
1,Not Like Us,Not Like Us,Kendrick Lamar,2024-05-04,USUG12400910,2,545.9,"323,703,884","28,113","174,597,137",...,3,67.0,"10,422,430",111.0,"7,780,028","28,444","6,623,075","1,118,279",NaN,1
2,i like the way you kiss me,I like the way you kiss me,Artemas,2024-03-19,QZJ842400387,3,538.4,"601,309,283","54,331","211,607,669",...,536,136.0,"36,321,847",172.0,"5,022,621","5,639","7,208,651","5,285,340",NaN,0
3,Flowers,Flowers - Single,Miley Cyrus,2023-01-12,USSM12209777,4,444.9,"2,031,280,633","269,802","136,569,078",...,"2,182",264.0,"24,684,248",210.0,"190,260,277","203,384",NaN,"11,822,942",NaN,0
4,Houdini,Houdini,Eminem,2024-05-31,USUG12403398,5,423.3,"107,034,922","7,223","151,469,874",...,1,82.0,"17,660,624",105.0,"4,493,884","7,006","207,179","457,017",NaN,1


In [79]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4595 entries, 0 to 4599
Data columns (total 29 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Track                       4595 non-null   object        
 1   Album Name                  4595 non-null   object        
 2   Artist                      4595 non-null   object        
 3   Release Date                4595 non-null   datetime64[ns]
 4   ISRC                        4595 non-null   object        
 5   All Time Rank               4595 non-null   object        
 6   Track Score                 4595 non-null   float64       
 7   Spotify Streams             4487 non-null   object        
 8   Spotify Playlist Count      4530 non-null   object        
 9   Spotify Playlist Reach      4528 non-null   object        
 10  Spotify Popularity          3796 non-null   float64       
 11  YouTube Views               4292 non-null   object        
 1

In [80]:
df_clean = df.copy()

# Alterar os tipos das colunas para os tipos adequados
numeric_columns = [
    'All Time Rank', 'Spotify Streams', 'Spotify Playlist Count', 'Spotify Playlist Reach',
    'YouTube Views', 'YouTube Likes', 'TikTok Posts', 'TikTok Likes', 'TikTok Views',
    'YouTube Playlist Reach', 'AirPlay Spins', 'SiriusXM Spins', 'Deezer Playlist Reach',
    'Pandora Streams', 'Pandora Track Stations', 'Soundcloud Streams', 'Shazam Counts'
]

for col in numeric_columns:
    df_clean[col] = df_clean[col].astype(str).str.replace(',', '').str.strip()
    df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')

#Padronizando colunas de texto
colunas_texto = [ 
    'Artist', 'Track', 'Album Name'
]

for col in colunas_texto:
    df_clean[col] = df_clean[col].str.strip().str.title()

In [81]:
# Remover a coluna 'TIDAL Popularity'
df_clean = df_clean.drop(columns=['TIDAL Popularity'])

# Removendo duplicatas
df_clean = df_clean.drop_duplicates()

In [82]:
# Substituir valores nulos por zero
df_sem_nulos = df_clean.copy()
colunas_substituir = [
    'Spotify Streams', 'Spotify Playlist Count', 'Spotify Playlist Reach',
    'Spotify Popularity', 'YouTube Views', 'YouTube Likes', 'YouTube Playlist Reach',
    'TikTok Posts', 'TikTok Likes', 'TikTok Views', 'Apple Music Playlist Count',
    'AirPlay Spins', 'SiriusXM Spins', 'Deezer Playlist Reach', 'Deezer Playlist Count',
    'Amazon Playlist Count', 'Pandora Streams', 'Pandora Track Stations', 
    'Soundcloud Streams', 'Shazam Counts'
]

df_sem_nulos[colunas_substituir] = df_sem_nulos[colunas_substituir].fillna(0)

# Verificando se ainda há valores nulos
df_sem_nulos.isnull().sum().sum()

np.int64(0)

### Salvando as dimensões em CSVs
Dimensões:
1. artista
2. música
3. ranking
4. stats nas plataformas

In [83]:
# --- 1. Dimensão Artista ---
dim_artist = df_sem_nulos[['Artist']].dropna().drop_duplicates().reset_index(drop=True)
dim_artist['artist_id'] = dim_artist.index + 1

# --- 2. Dimensão Track ---
dim_track = df_sem_nulos[['Track', 'Album Name', 'Explicit Track', 'Release Date', 'Artist']].drop_duplicates()
dim_track = dim_track.merge(dim_artist, on='Artist', how='left') # Adicionando o ID do artista
dim_track['track_id'] = dim_track.index + 1

# --- 3. Dimensão Rank ---
dim_rank = df_sem_nulos[['All Time Rank', 'Track']].drop_duplicates()
dim_rank['rank_id'] = dim_rank.index + 1

# --- 4. Dimensão Stats ---
stats_columns = [
    'Track', 'Spotify Streams', 'Spotify Playlist Count', 'Spotify Playlist Reach',
    'YouTube Views', 'YouTube Likes', 'TikTok Posts', 'TikTok Likes', 'TikTok Views',
    'YouTube Playlist Reach', 'AirPlay Spins', 'SiriusXM Spins',
    'Deezer Playlist Count', 'Deezer Playlist Reach', 'Amazon Playlist Count',
    'Pandora Streams', 'Pandora Track Stations', 'Soundcloud Streams', 'Shazam Counts',
    'Spotify Popularity', 'Apple Music Playlist Count'
]

dim_stats = df_sem_nulos[stats_columns].drop_duplicates()
dim_stats = dim_stats.merge(dim_track[['Track', 'track_id']], on='Track', how='left')  # Adicionando o ID 
dim_stats.drop(columns=['Track'], inplace=True)

In [84]:
# carregando as dimensões em arquivos CSV na pasta dimensoes

import os

os.makedirs('dimensoes', exist_ok=True)

dim_artist.to_csv('dimensoes/dim_artist.csv', index=False)
dim_track.to_csv('dimensoes/dim_track.csv', index=False)
dim_rank.to_csv('dimensoes/dim_rank.csv', index=False)
dim_stats.to_csv('dimensoes/dim_stats.csv', index=False)

print("Arquivos CSV gerados na pasta 'dimensoes':")
for file in os.listdir('dimensoes'):
    print(file)

Arquivos CSV gerados na pasta 'dimensoes':
dim_artist.csv
dim_rank.csv
dim_stats.csv
dim_track.csv
